## Example yy_fmri_kit pipeline for one participant
DICOM → BIDS → fMRIPrep → Build group mask → Smoothing and Denoising → Apply group mask and compute z-scores → Parcellation  → HRF shift → ISC / IS-RSA
- Test run for one subject end-to-end.

### Step 1: Convert Dicoms to Bids

In [ ]:
from pathlib import Path
import pandas as pd
from yy_fmri_kit.helpers.preproc.dicom2bids import (
    get_dicom_info,
    convert_dicom_to_bids,
    validate_bids_dataset
)
from yy_fmri_kit.static.preproc.config import (
    HEUDICONV_CMD_TEMPLATE,
    BIDS_VALIDATOR_CMD_TEMPLATE
)

In [ ]:
# Configure paths - adjust as needed
DICOM_ROOT = Path("/path/to/dicom_data")
BIDS_ROOT  = Path("/path/to/bids_dataset")
HEURISTIC  = Path("/path/to/heuristics.py")

Run conversion and validation:

In [ ]:
# 1) Make TSV + read it
tsv = get_dicom_info(DICOM_ROOT)
df = pd.read_csv(tsv, sep="\t")

# 2) Filter to a single subject (e.g., first row)
# change as needed
df = df

# 3) Convert
convert_dicom_to_bids(
    df=df,
    heuristic=HEURISTIC,
    bids_path=BIDS_ROOT,
    heudiconv_template=HEUDICONV_CMD_TEMPLATE,
    overwrite=True,
)

# 4) Validate
ok = validate_bids_dataset(BIDS_ROOT, BIDS_VALIDATOR_CMD_TEMPLATE)
print("VALID:", ok)

### Step 2: Run fmriprep

In [ ]:
import importlib
from pathlib import Path
from yy_fmri_kit.helpers.preproc import fmriprep
importlib.reload(fmriprep)
from yy_fmri_kit.static.preproc.config import FMRIPREP_DOCKER_TEMPLATE

In [ ]:
# Configure paths - adjust as needed
FMRIPREP_DERIV     = Path("path/to/fmriprep_derivatives")
BIDS_ROOT          = Path("path/to/bids")      
WORK               = Path("/path/to/fmri_analysis/work")
FREESURFER_LICENSE = Path("/path/to/fmri_analysis/FreeSurfer/license.txt")
SUBJECT            = ["sub-5", "sub-6"]  # list of subjects to process

In [ ]:
# TODO: Modify run_fmriprep to accept None or list of subjects

Run fmriprep:

In [ ]:
fmriprep.run_fmriprep(
    bids_root=BIDS_ROOT,
    derivatives_root=FMRIPREP_DERIV,
    work_dir=WORK,
    fs_license_file=FREESURFER_LICENSE,
    subject_label=SUBJECT, # if no identifier, runs all subjects
    template=FMRIPREP_DOCKER_TEMPLATE,
    spaces=["MNI152NLin2009cAsym:res-2"],
    nthreads=6, # adjust as needed
)

## Step 3: Smooth and Denoise

In [ ]:
import importlib
from pathlib import Path
from yy_fmri_kit.helpers.preproc import group_mask
importlib.reload(group_mask)
from yy_fmri_kit.helpers.find_files import find_brain_mask
from yy_fmri_kit.helpers.preproc.denoising import run_subject_denoising

In [ ]:
# Configure paths - adjust as needed
FMRIPREP_DERIV = Path("path/to/fmriprep_derivatives_directory")
DENOISED_DERIV = Path("path/to/denoised_derivatives_directory")
GROUPMASK_PATH = Path("path/to/group_mask.nii.gz")
# Find brain mask - auto-discover if needed
BRAINMASK_PATH = find_brain_mask(FMRIPREP_DERIV, sub=None)

2.1. Compute group mask to later exclude outlier voxels:

In [ ]:
# TODO: Still debugging build_group_mask

group_mask.build_group_mask(
    fmriprep_root=FMRIPREP_DERIV,
    brain_mask_img=BRAINMASK_PATH,
    groupmask_out_path=GROUPMASK_PATH,
    pass_fraction=0.90,                 # ≥ 90% of subjects
    tsnr_percentile=40.0,               # keep the top 60% most stable voxels (per subject), adaptive
    fixed_thresh=None,
    verbose=True,
)

2.2. Run denoising on one subject:

In [ ]:
run_subject_denoising(
    fmriprep_derivatives_root=FMRIPREP_DERIV,
    denoised_root=DENOISED_DERIV,
    sub_label="sub-1",
    tr=1.0,
    fwhm=4.0,
    spike_cutoff=5.0,
)

Possible - denoising all subjects at once:

In [ ]:
from yy_fmri_kit.helpers.preproc import denoising
importlib.reload(denoising)

subjects = ["sub-4", "sub-5", "sub-6"] # if not provided or None, runs all fmriprep subjects

In [ ]:
denoising.run_group_denoising(
    fmriprep_derivatives_root=FMRIPREP_DERIV,
    denoised_root=DENOISED_DERIV,
    tr=1.0,
    fwhm=4.0,
    spike_cutoff=5.0,
    subjects=subjects
)

Optional: Visualize one denoised voxel to inspect changes

In [ ]:
import matplotlib.pyplot as plt
from yy_fmri_kit.helpers.preproc.visualization import plot_voxelwise_std_hist

In [ ]:
PRE_BOLD_PATH = Path("/path/to/your/fmriprep_bold.nii.gz")
POST_BOLD_PATH = Path("/path/to/your/post_denoised_bold.nii.gz")

In [ ]:
# Standard deviation histograms before and after denoising
plot_voxelwise_std_hist(
    pre_bold=PRE_BOLD_PATH,
    post_bold=POST_BOLD_PATH,
    bins=100,
    log=True,
    title="Voxelwise std: pre vs post denoising"
)
plt.show()

2.3. Apply group mask on denoised data and compute z-scores:

In [ ]:
# TODO: test and debug after having build_group_mask working

In [ ]:
from yy_fmri_kit.helpers.preproc.group_mask import apply_group_mask_for_all

In [ ]:
# Configure paths - adjust as needed
MASKED_DERIV = Path("path/to/masked_derivatives_directory")

In [ ]:
apply_group_mask_for_all(
    fmriprep_root=FMRIPREP_DERIV,
    denoised_root=DENOISED_DERIV,
    mask_path=GROUPMASK_PATH,
    masked_derivatives_root=MASKED_DERIV,
    zscore=True,
)


## Step 4: Time-shift each BOLD time series

Using first auditory BOLD peak in each scan.

In [ ]:
# TODO: Create a module to find, extract and implement time shift on each subjects and all subjects

In [ ]:
import importlib
from pathlib import Path
import nibabel as nib
from yy_fmri_kit.helpers.preproc import time_shift
importlib.reload(time_shift)
from yy_fmri_kit.helpers.preproc.time_shift import time_shift_all_denoised, build_default_auditory_roi

In [ ]:
DERIV     = Path("/path/to/fmri_analysis/data/derivatives/denoised")
TIMESHIFT = Path("/path/to/fmri_analysis/data/derivatives/denoised_ts")
ROI       = Path("/path/to/fmri_analysis/data/derivatives/roi_masks")

In [ ]:
# Before time shifting, build the default auditory ROI mask -> IGNORE if you already have it
# It is best to insure that the ROI is built from the same reference image as the BOLD data
t1   = nib.load("/path/to/fmri_analysis/data/derivatives/roi_masks/MNI152_T1_2mm_brain.nii")
bold = nib.load("/path/to/fmri_analysis/data/derivatives/denoised/sub-1/func/sub-1_ses-202505251228_task-AntiRight_space-MNI152NLin2009cAsym_res-2_desc-preproc_desc-nltoolsClean_bold.nii.gz")

print("T1 shape:", t1.shape)
print("BOLD shape:", bold.shape[:3])
print("T1 affine:\n", t1.affine)
print("BOLD affine:\n", bold.affine)


In [ ]:
# If matching -> build the ROI
REF = Path("/path/to/fmri_analysis/data/derivatives/roi_masks/MNI152_T1_2mm_brain.nii")
roi_path = build_default_auditory_roi(
    ref_img_path=REF,
    out_dir=ROI,
    overwrite=False,
    radius_mm=10.0,
)
print("Returned path:", roi_path)
print("Is file?", Path(roi_path).is_file())
print("Exists?", Path(roi_path).exists())

In [ ]:
shifted_paths, lag_info = time_shift_all_denoised(
    derivatives_dir=DERIV,
    roi_mask_img=roi_path,
    use_default_auditory_roi=False,
    default_roi_out_dir=ROI,
    out_root=TIMESHIFT,
    max_lag_tr=4,
)


In [ ]:
# TODO: compute audio envelope to estimate the fit of the shift

## Step 5: Parcellation

Using TemplateFlow

In [ ]:
from pathlib import Path
import importlib
from yy_fmri_kit.helpers import find_files
importlib.reload(find_files)
from yy_fmri_kit.helpers.preproc import parcellation
importlib.reload(parcellation)

In [ ]:
DENOISED_DERIV    = Path("/path/to/fmri_analysis/data/derivatives/denoised_ts")
PARCELLATED_DERIV = Path("/path/to/fmri_analysis/data/derivatives/parcellated_ts")
SUBJECTS         = ["sub-1", "sub-2", "sub-4", "sub-5", "sub-6"]  # list of subjects to process, or None for all

In [ ]:
# Parcellating denoised data
parcellation.parcellate_group(
    denoised_root=DENOISED_DERIV,
    out_root=PARCELLATED_DERIV,
    subjects=None,
    tf_template="MNI152NLin2009cAsym",
    tf_atlas="Schaefer2018",
    tf_desc="400Parcels7Networks",
    tf_resolution=2)